<a href="https://colab.research.google.com/github/PedroTechy/CarrisInsight/blob/streaming_development/spark_jobs/extract_carris_vehicles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Authenticate with Google Cloud


In [1]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=6Jpx2FkIAGx6rqiZL50YALX4pYR30u&prompt=consent&token_usage=remote&access_type=offline&code_challenge=1D7mGMuiu2RULIaB7St1v3NjLIQFOjq-82i3zUC7veA&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AanRRrvfLaTrqkac_sXQx2ZH2VR_BfcBaya2Zr-32X9I9S-J-gU9MtvnfC6270C0CEl9_g

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

# Step 2: Install Spark and BigQuery connector

In [2]:
# Install OpenJDK 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark from a reliable source
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

# Extract the downloaded Spark tarball
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

!rm -rf spark-3.2.1-bin-hadoop3.2.tgz # No longer needed

# Install the GCS Connector
!rm -rf /content/spark-3.2.1-bin-hadoop3.2/jars/gcs-connector-hadoop3-latest.jar
!wget -q https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar -P /content/spark-3.2.1-bin-hadoop3.2/jars/

# Install the BigQuery Connector
!rm -rf /content/spark-3.2.1-bin-hadoop3.2/jars/spark-bigquery-with-dependencies_2.12-0.29.0.jar
!wget -q https://storage.googleapis.com/spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.29.0.jar -P /content/spark-3.2.1-bin-hadoop3.2/jars/

# Install Findspark
!pip install -q findspark


# Step 3: Set environment variables for Spark and Java

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/.config/application_default_credentials.json"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

# Step 4: Initialize Spark session

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GCS to BigQuery Streaming") \
    .config("spark.jars", "/content/spark-3.2.1-bin-hadoop3.2/jars/gcs-connector-hadoop3-latest.jar,/content/spark-3.2.1-bin-hadoop3.2/jars/spark-bigquery-with-dependencies_2.12-0.29.0.jar") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .getOrCreate()

print("Spark Session successfully created!")


Spark Session successfully created!


# Step 5: Define GCS input path and output BigQuery table

In [ ]:
input_path = "gs://edit-de-project-streaming-data/carris-vehicles"
output_table = "data-eng-dev-437916.data_eng_project_group3_raw"

# Step 6: Read streaming data from GCS

In [6]:
# Define the schema for your JSON files
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType

schema = StructType([
    StructField("bearing", FloatType(), True),
    StructField("block_id", StringType(), True),
    StructField("current_status", StringType(), True),
    StructField("id", StringType(), True),
    StructField("lat", FloatType(), True),
    StructField("line_id", StringType(), True),
    StructField("lon", FloatType(), True),
    StructField("pattern_id", StringType(), True),
    StructField("route_id", StringType(), True),
    StructField("schedule_relationship", StringType(), True),
    StructField("shift_id", StringType(), True),
    StructField("speed", FloatType(), True),
    StructField("stop_id", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("trip_id", StringType(), True)
])

# Create the streaming DataFrame with the defined schema
streaming_df = spark.readStream \
    .format("json") \
    .schema(schema) \
    .load(input_path)

In [90]:
def transform_data(df, batch_id):

  transformed_df = df.withColumn("timestamp", to_timestamp(from_unixtime("timestamp")))

  return transformed_df.write.mode("append").format("parquet").save("content/lake/silver/vehicles/data")



In [ ]:
def haversine(lat1, lon1, lat2, lon2): # Earth radius in kilometers R = 6371.0# Convert latitude and longitude from degrees to radians lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1) lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2) # Differences delta_lat = lat2_rad - lat1_rad delta_lon = lon2_rad - lon1_rad # Haversine formula a = math.sin(delta_lat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2)**2 c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a)) # Distance distance = R * c return distance # Example usage lat1, lon1 = 52.2296756, 21.0122287  # Warsaw lat2, lon2 = 41.8919300, 12.5113300  # Rome distance = haversine(lat1, lon1, lat2, lon2)

In [92]:
df = (spark.readStream.option("maxFilesPerTrigger", 1)
    .format("json")
    .schema(schema)
    .load(input_path))


query = (df.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/lake/silver/vehicles_checkpoint') #using a common checkpoint for both messages streams.
.trigger(processingTime='5 seconds')
.foreachBatch(transform_data)
.start()
)

In [95]:
df = spark.read.format("parquet").load("content/lake/silver/vehicles/data")


In [97]:
df.show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|  261.0|20250116-64010206...| IN_TRANSIT_TO|44|12066|38.523514|   4434|-8.893788|  4434_0_1|  4434_0|            SCHEDULED|112440234560|      0.0| 160105|2025-01-16 12:16:42|4434_0_1|2700|120...|
|  260.0|20250116-64010318...| IN_TRANSIT_TO|44|12574|38.570927|   4562|-8.885346|  4562_0_1|  4562_0|            SCHEDULED|111060234560|      0.0| 130663|2025-01-16 12:16:54|4562_0_1|2700|114...|
|  299.0|202501

In [87]:
import tempfile

import time

_ = spark.sql("DROP TABLE IF EXISTS my_table2")


    # Create a table with Rate source.
    query = (spark.readStream.option("maxFilesPerTrigger", 1)
    .format("json")
    .schema(schema)
    .load(input_path).withColumn("timestamp", to_timestamp(from_unixtime("timestamp")))
    .writeStream
             .toTable(

            "vehicles_table",

            queryName='read_query',

            outputMode="append",

            format='parquet',

            checkpointLocation=d))

    time.sleep(10)

    query.stop()



In [88]:
spark.read.table("vehicles_table").show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+----------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id| timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+----------+--------------------+
|  183.0|20250116-64010165...| IN_TRANSIT_TO|44|12745| 38.76761|   4720|-9.100393|  4720_0_1|  4720_0|            SCHEDULED|113260234560|      0.0| 060011|1737026213|4720_0_1|2700|111...|
|   58.0|20250116-64010273...| IN_TRANSIT_TO|44|12557| 38.57138|   4641| -9.03899|  4641_0_2|  4641_0|            SCHEDULED|111500234560|      0.0| 150013|1737026203|4641_0_2|2700|102...|
|   12.0|20250116-64010076...|    STOPPED_AT|44|12092| 38.70

In [86]:
with tempfile.TemporaryDirectory() as d:
    # Create a stream from the input data
    df = (spark.readStream
          .option("maxFilesPerTrigger", 1)
          .format("json")
          .schema(schema)  # Define the schema for the input data
          .load(input_path))

    # Apply transformation to the DataFrame
    transformed_df = (df
                       .withColumn("timestamp", to_timestamp(from_unixtime("timestamp"))))  # Example transformation

    # Write the transformed data to the table
    query = (transformed_df.writeStream(
               "vehicles_table",

            queryName='read_query',

            outputMode="append",

            format='parquet',

            checkpointLocation=d ) ) # Define the checkpoint location

    # Sleep for a while to let the stream process some data
    time.sleep(10)

    # Stop the stream query after processing
    query.stop()


TypeError: 'DataStreamWriter' object is not callable

In [48]:
# prompt: convert the column timestamp to timestamp

from pyspark.sql.functions import from_unixtime, to_timestamp


In [ ]:
# prompt: group by vehicle in 2 minutes buckets

from pyspark.sql.functions import window, from_unixtime, col, to_timestamp

# Assuming 'vehicles_table' is your table with the vehicle data

# Read the data
vehicle_data = spark.read.parquet("vehicles_table")

# Convert the timestamp to a timestamp type and adjust timezone
vehicle_data = vehicle_data.withColumn("timestamp_readable", to_timestamp(from_unixtime(col("timestamp") / 1000)))

# Group by vehicle and 2-minute intervals
grouped_vehicle_data = vehicle_data.groupBy(
    "id", window("timestamp_readable", "2 minutes")
).count()


# Show the results
grouped_vehicle_data.show(truncate=False)

# Optional: Write the grouped data to a new location
# grouped_vehicle_data.write.mode("overwrite").parquet("grouped_vehicles_data")

In [50]:
  spark.read.table("vehicles_table").withColumn("timestamp", to_timestamp(from_unixtime("timestamp"))).show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|  183.0|20250116-64010165...| IN_TRANSIT_TO|44|12745| 38.76761|   4720|-9.100393|  4720_0_1|  4720_0|            SCHEDULED|113260234560|      0.0| 060011|2025-01-16 11:16:53|4720_0_1|2700|111...|
|   58.0|20250116-64010273...| IN_TRANSIT_TO|44|12557| 38.57138|   4641| -9.03899|  4641_0_2|  4641_0|            SCHEDULED|111500234560|      0.0| 150013|2025-01-16 11:16:43|4641_0_2|2700|102...|
|   12.0|202501

In [33]:
spark.read.table("vehicles_table").write.format('bigquery') \
  .option('table', output_table) \
  .option("createDisposition", "CREATE_IF_NEEDED") \
  .option("temporaryGcsBucket", "edit-data-eng-project-group3/streaming_data/temp-bucket") \
  .mode("overwrite")
  .save()

Py4JJavaError: An error occurred while calling o274.save.
: com.google.cloud.bigquery.connector.common.BigQueryConnectorException: Failed to write to BigQuery
	at com.google.cloud.spark.bigquery.write.BigQueryWriteHelper.writeDataFrameToBigQuery(BigQueryWriteHelper.java:110)
	at com.google.cloud.spark.bigquery.write.BigQueryDeprecatedIndirectInsertableRelation.insert(BigQueryDeprecatedIndirectInsertableRelation.java:43)
	at com.google.cloud.spark.bigquery.write.CreatableRelationProviderHelper.createRelation(CreatableRelationProviderHelper.java:54)
	at com.google.cloud.spark.bigquery.BigQueryRelationProvider.createRelation(BigQueryRelationProvider.scala:107)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:73)
	at org.apac

# Step 7: Write streaming data to BigQuery with auto-table creation

In [7]:
streaming_query = streaming_df.writeStream \
    .format("bigquery") \
    .option("table", output_table) \
    .option("checkpointLocation", "gs://edit-data-eng-project-group3/streaming_data/checkpoints") \
    .option("temporaryGcsBucket", "gs://edit-data-eng-project-group3/streaming_data/temp-bucket") \
    .option("writeDisposition", "WRITE_APPEND") \
    .option("createDisposition", "CREATE_IF_NEEDED") \
    .outputMode("append") \
    .start()

streaming_query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [8]:
streaming_query.stop()